In [1]:
"""
    Edit obj_func, excesscurtail, req curtail and checkvalidity

    Add Values on Antonio and Martin Base Values Below (Load Capacity, Max Off, Min On)
    L0: 320, 4, 2
    L1: 200, 4, 2
    L2: 80, 4, 2
    L3: 84, 4, 2
    L4: 100, 4, 2
    L5: 160, 4, 2
    L6: 100, 3, 1
    L7: 60, 3, 1
    L8: 200, 3, 1
    L9: 40, 4, 1
    L10: 40, 3, 1
    L11: 72, 3, 3
    L12: 140, 4, 3
    L13: 80, 3, 3
    L14: 40, 3, 3
    L15: 180, 3, 3
    L16: 180, 4, 2
    L17: 180, 4, 2
    L18: 60, 4, 2
    L19: 50, 4, 2
    
    Req Curtailment: (T0, T1, T2... T15)
    110, 230, 455, 680, 770, 800, 750, 640, 590, 610, 660, 680, 570, 410, 230, 135
"""



# Import modules
import numpy as np
import timeit

# Import sphere function as objective function
from pyswarms.utils.functions.single_obj import sphere as f

# Import backend modules
import pyswarms.backend as P
from pyswarms.backend.topology import Star
#import pyswarms as ps
#import pyswarms.discrete.binary as PSO

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
my_topology = Star() # The Topology Class
my_options = {'c1': 2, 'c2': 2, 'w': 2}
c1 = 2
c2 = 2
w = 2
#set dimension to 20 loads x 64 timeslots = 1280
my_swarm = P.create_swarm(n_particles=30, dimensions=1280, binary=True, discrete=True, options=my_options) # The Swarm Class

#Initialize cost to array of inf for max value, to_append is manually set below. Create temp storade for pbest cost as well
x = 0
temp_pbest_cost = []
to_append = np.array([float('inf')])

for x in range(my_swarm.n_particles):
    my_swarm.pbest_cost = np.append(my_swarm.pbest_cost, to_append)
    my_swarm.current_cost = np.append(my_swarm.current_cost, to_append)
    temp_pbest_cost = np.append(temp_pbest_cost, to_append)




#Printing Values for debugging
#print('The following are the attributes of our swarm: {}'.format(my_swarm.__dict__.keys()))
#print("# of particles:",my_swarm.n_particles)
#print("dimensions:",my_swarm.dimensions)
#print("pos \n",my_swarm.position)
#print("velo \n",my_swarm.velocity)
#print("current cost \n",my_swarm.current_cost)
#print("pbest pos \n",my_swarm.pbest_pos)
#print("pbest cost \n",my_swarm.pbest_cost)
#print("best cost \n",my_swarm.best_cost)

In [3]:
#Position computation, BPSO computation for position

def _compute_position(swarm):
    for i in range(swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            if swarm.velocity[i][j] < np.random.uniform(low=0.0, high=1.0):
                swarm.position[i][j] = 0
            else:
                swarm.position[i][j] = 1
    return swarm.position

In [4]:
#Velocity computation, BPSO computation for velocity

def _compute_velocity(swarm,c1,c2,w):
    a = np.multiply(swarm.velocity,w)
    b = np.multiply(np.subtract(swarm.pbest_pos,swarm.position),c1*np.random.uniform(low=0.0, high=1.0))
    c = np.multiply(np.subtract(swarm.best_pos,swarm.position),c2*np.random.uniform(low=0.0, high=1.0))
    swarm.velocity = a + b + c

    return 1 / (1 + np.exp(-swarm.velocity))

In [5]:
#Taken from pyswarms.backend.operators
def _compute_pbest(swarm,temp_pbest_cost):
    try:
        # Infer dimensions from positions
        dimensions = swarm.dimensions
        
        # Create a 1-D and 2-D mask based from comparisons
        # Creating 1-D mask
        mask_cost = swarm.current_cost < temp_pbest_cost
        #print("maskcost:\n",mask_cost)
        
        # Extension of 1-D mask to 2-D mask
        mask_pos = np.repeat(mask_cost[:, np.newaxis], dimensions, axis=1)
        #print("maskpos:\n",mask_pos)
        
        # Apply masks, copy into 2nd variable if 3rd variable fits conditions
        new_pbest_pos = np.where(~mask_pos, swarm.pbest_pos, swarm.position)
        new_pbest_cost = np.where(~mask_cost, temp_pbest_cost, swarm.current_cost)
        
        #print("swarm current pos:",swarm.position)
        #print("new swarm pbest pos:",new_pbest_pos)
        #print("new swarm pbest cost:",new_pbest_cost)
        
    except AttributeError:
        rep.logger.exception(
            "Please pass a Swarm class. You passed {}".format(type(swarm))
        )
        raise
    else:
        return (new_pbest_pos, new_pbest_cost)

In [6]:
import numpy as np
#create array for SGE
def SGEgenerate(mode):
    #amount of self generated energy per load
    self0 = 15/4
    self1 = 7/4
    self2 = 2/4
    self3 = 5/4
    self4 = 8/4
    self5 = 9/4
    self6 = 6/4
    self7 = 3/4
    self8 = 17/4
    self9 = 3/4
    self10 = 2/4
    self11 = 1/4
    self12 = 6/4
    self13 = 4/4
    self14 = 2/4
    self15 = 13/4
    self16 = 12/4
    self17 = 9/4
    self18 = 2/4
    self19 = 3/4
    
    #SGE is off
    if mode == 0:
        SGE = np.zeros(1280)
        
    #SGE is constant
    if mode == 1:
        SGE = []
        T0 = np.full(64,self0)
        T1 = np.full(64,self1)
        T2 = np.full(64,self2)
        T3 = np.full(64,self3)
        T4 = np.full(64,self4)
        T5 = np.full(64,self5)
        T6 = np.full(64,self6)
        T7 = np.full(64,self7)
        T8 = np.full(64,self8)
        T9 = np.full(64,self9)
        T10 = np.full(64,self10)
        T11 = np.full(64,self11)
        T12 = np.full(64,self12)
        T13 = np.full(64,self13)
        T14 = np.full(64,self14)
        T15 = np.full(64,self15)
        T16 = np.full(64,self16)
        T17 = np.full(64,self17)
        T18 = np.full(64,self18)
        T19 = np.full(64,self19)
        
        SGE = np.append(SGE,[T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19])
    
    #SGE is random value but on
    if mode == 2:
        SGE = []
        T0 = np.random.randint(self0+1,size=64)
        T1 = np.random.randint(self1+1,size=64)
        T2 = np.random.randint(self2+1,size=64)
        T3 = np.random.randint(self3+1,size=64)
        T4 = np.random.randint(self4+1,size=64)
        T5 = np.random.randint(self5+1,size=64)
        T6 = np.random.randint(self6+1,size=64)
        T7 = np.random.randint(self7+1,size=64)
        T8 = np.random.randint(self8+1,size=64)
        T9 = np.random.randint(self9+1,size=64)
        T10 = np.random.randint(self10+1,size=64)
        T11 = np.random.randint(self11+1,size=64)
        T12 = np.random.randint(self12+1,size=64)
        T13 = np.random.randint(self13+1,size=64)
        T14 = np.random.randint(self14+1,size=64)
        T15 = np.random.randint(self15+1,size=64)
        T16 = np.random.randint(self16+1,size=64)
        T17 = np.random.randint(self17+1,size=64)
        T18 = np.random.randint(self18+1,size=64)
        T19 = np.random.randint(self19+1,size=64)
        
        SGE = np.append(SGE,[T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19])
    
    #SGE is fixed value but on or off randomly
    if mode == 3:
        SGE = []
        T0 = np.full(64,self0)
        T1 = np.full(64,self1)
        T2 = np.full(64,self2)
        T3 = np.full(64,self3)
        T4 = np.full(64,self4)
        T5 = np.full(64,self5)
        T6 = np.full(64,self6)
        T7 = np.full(64,self7)
        T8 = np.full(64,self8)
        T9 = np.full(64,self9)
        T10 = np.full(64,self10)
        T11 = np.full(64,self11)
        T12 = np.full(64,self12)
        T13 = np.full(64,self13)
        T14 = np.full(64,self14)
        T15 = np.full(64,self15)
        T16 = np.full(64,self16)
        T17 = np.full(64,self17)
        T18 = np.full(64,self18)
        T19 = np.full(64,self19)
        
        SGE = np.append(SGE,[T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19])
        rand = np.random.randint(2,size=1280)
        SGE = np.multiply(SGE,rand)
        
    return SGE

In [7]:
def merge4arr(a,b,c,d,totallen):
    
    arr = [None]*totallen
    arr[::4] = a
    arr[1::4] = b
    arr[2::4] = c
    arr[3::4] = d
    
    return arr

In [8]:
#penalty for curtailing more than necessary (lighter penalty)
def overcurtail(x):
    
    #timeslotrange = 1280 elements per particle divided by 20 loads
    timeslotrange = 64
    penaltyvalue = 400000   
    totalpenalty = 0
    
    #divide array into 20 loads x 64 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    #invert array 1 and 0, count 1s to find number of 0s
    inv = np.ones(64, dtype=int)
    L0 = np.subtract(inv,L0)
    L1 = np.subtract(inv,L1)
    L2 = np.subtract(inv,L2)
    L3 = np.subtract(inv,L3)
    L4 = np.subtract(inv,L4)
    L5 = np.subtract(inv,L5)
    L6 = np.subtract(inv,L6)
    L7 = np.subtract(inv,L7)
    L8 = np.subtract(inv,L8)
    L9 = np.subtract(inv,L9)
    L10 = np.subtract(inv,L10)
    L11 = np.subtract(inv,L11)
    L12 = np.subtract(inv,L12)
    L13 = np.subtract(inv,L13)
    L14 = np.subtract(inv,L14)
    L15 = np.subtract(inv,L15)
    L16 = np.subtract(inv,L16)
    L17 = np.subtract(inv,L17)
    L18 = np.subtract(inv,L18)
    L19 = np.subtract(inv,L19)
    
    #set maximum off time per load here
    limitL0 = 4*4
    limitL1 = 4*4
    limitL2 = 4*4
    limitL3 = 4*4
    limitL4 = 4*4
    limitL5 = 4*4
    limitL6 = 3*4
    limitL7 = 3*4
    limitL8 = 3*4
    limitL9 = 4*4
    limitL10 = 3*4
    limitL11 = 3*4
    limitL12 = 4*4
    limitL13 = 3*4
    limitL14 = 3*4
    limitL15 = 3*4
    limitL16 = 4*4
    limitL17 = 4*4
    limitL18 = 4*4
    limitL19 = 4*4
    
    #count for off time here, add 1 penalty value if violated
    if np.sum(L0) > limitL0:
        totalpenalty += 1
    if np.sum(L1) > limitL1:
        totalpenalty += 1
    if np.sum(L2) > limitL2:
        totalpenalty += 1
    if np.sum(L3) > limitL3:
        totalpenalty += 1
    if np.sum(L4) > limitL4:
        totalpenalty += 1
    if np.sum(L5) > limitL5:
        totalpenalty += 1
    if np.sum(L6) > limitL6:
        totalpenalty += 1
    if np.sum(L7) > limitL7:
        totalpenalty += 1
    if np.sum(L8) > limitL8:
        totalpenalty += 1
    if np.sum(L9) > limitL9:
        totalpenalty += 1
    if np.sum(L10) > limitL10:
        totalpenalty += 1
    if np.sum(L11) > limitL11:
        totalpenalty += 1
    if np.sum(L12) > limitL12:
        totalpenalty += 1
    if np.sum(L13) > limitL13:
        totalpenalty += 1
    if np.sum(L14) > limitL14:
        totalpenalty += 1
    if np.sum(L15) > limitL15:
        totalpenalty += 1
    if np.sum(L16) > limitL16:
        totalpenalty += 1
    if np.sum(L17) > limitL17:
        totalpenalty += 1
    if np.sum(L18) > limitL18:
        totalpenalty += 1
    if np.sum(L19) > limitL19:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [9]:
#penalty for undercurtailing (should be high multiplier to ensure output is not all zero)
def undercurtail(x):
    
    #timeslotrange = 1280 elements per particle divided by 20 loads
    timeslotrange = 64
    penaltyvalue = 10000
    totalpenalty = 0
    
    #divide array into 20 loads x 64 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    #set minimum on time per load here
    limitL0 = 2*4
    limitL1 = 2*4
    limitL2 = 2*4
    limitL3 = 2*4
    limitL4 = 2*4
    limitL5 = 2*4
    limitL6 = 1*4
    limitL7 = 1*4
    limitL8 = 1*4
    limitL9 = 1*4
    limitL10 = 1*4
    limitL11 = 3*4
    limitL12 = 3*4
    limitL13 = 3*4
    limitL14 = 3*4
    limitL15 = 3*4
    limitL16 = 2*4
    limitL17 = 2*4
    limitL18 = 2*4
    limitL19 = 2*4
    
    #count for ontime here, add 1 penalty value if violated
    if np.sum(L0) < limitL0:
        totalpenalty += 1
    if np.sum(L1) < limitL1:
        totalpenalty += 1
    if np.sum(L2) < limitL2:
        totalpenalty += 1
    if np.sum(L3) < limitL3:
        totalpenalty += 1
    if np.sum(L4) < limitL4:
        totalpenalty += 1
    if np.sum(L5) < limitL5:
        totalpenalty += 1
    if np.sum(L6) < limitL6:
        totalpenalty += 1
    if np.sum(L7) < limitL7:
        totalpenalty += 1
    if np.sum(L8) < limitL8:
        totalpenalty += 1
    if np.sum(L9) < limitL9:
        totalpenalty += 1
    if np.sum(L10) < limitL10:
        totalpenalty += 1
    if np.sum(L11) < limitL11:
        totalpenalty += 1
    if np.sum(L12) < limitL12:
        totalpenalty += 1
    if np.sum(L13) < limitL13:
        totalpenalty += 1
    if np.sum(L14) < limitL14:
        totalpenalty += 1
    if np.sum(L15) < limitL15:
        totalpenalty += 1
    if np.sum(L16) < limitL16:
        totalpenalty += 1
    if np.sum(L17) < limitL17:
        totalpenalty += 1
    if np.sum(L18) < limitL18:
        totalpenalty += 1
    if np.sum(L19) < limitL19:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [10]:
#penalty for not meeting required curtailment per timeslot
def reqcurtail(x,SGE):
    penaltyvalue = 1200000
    totalpenalty = 0
    
    #set requirement for each TS in kW here
    TS0limit = 110/4
    TS1limit = 220/4
    TS2limit = 455/4
    TS3limit = 680/4
    TS4limit = 770/4
    TS5limit = 800/4
    TS6limit = 750/4
    TS7limit = 640/4
    TS8limit = 590/4
    TS9limit = 610/4
    TS10limit = 660/4
    TS11limit = 680/4
    TS12limit = 570/4
    TS13limit = 410/4
    TS14limit = 230/4
    TS15limit = 135/4
    
    #set consumption for each load in kW here
    ctotal = []
    cons0 = 320/4
    cons1 = 200/4
    cons2 = 80/4
    cons3 = 84/4
    cons4 = 100/4
    cons5 = 160/4
    cons6 = 100/4
    cons7 = 60/4
    cons8 = 200/4
    cons9 = 40/4
    cons10 = 40/4
    cons11 = 72/4
    cons12 = 140/4
    cons13 = 80/4
    cons14 = 40/4
    cons15 = 180/4
    cons16 = 180/4
    cons17 = 160/4
    cons18 = 60/4
    cons19 = 50/4
    
    c0 = np.full(64,cons0)
    c1 = np.full(64,cons1)
    c2 = np.full(64,cons2)
    c3 = np.full(64,cons3)
    c4 = np.full(64,cons4)
    c5 = np.full(64,cons5)
    c6 = np.full(64,cons6)
    c7 = np.full(64,cons7)
    c8 = np.full(64,cons8)
    c9 = np.full(64,cons9)
    c10 = np.full(64,cons10)
    c11 = np.full(64,cons11)
    c12 = np.full(64,cons12)
    c13 = np.full(64,cons13)
    c14 = np.full(64,cons14)
    c15 = np.full(64,cons15)
    c16 = np.full(64,cons16)
    c17 = np.full(64,cons17)
    c18 = np.full(64,cons18)
    c19 = np.full(64,cons19)
    ctotal = np.append(ctotal,[c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19])
    
    #consumption = total req - consumed energy
    consumption = np.subtract(ctotal,SGE)
    
    #cut particles into timeslots here, from TS0 to TS15. Note: 1280 = 20 loads x 64 timeslots
    TS0a = x[0:1280:64]
    TS0b = x[1:1280:64]
    TS0c = x[2:1280:64]
    TS0d = x[3:1280:64]
    TS1a = x[4:1280:64]
    TS1b = x[5:1280:64]
    TS1c = x[6:1280:64]
    TS1d = x[7:1280:64]
    TS2a = x[8:1280:64]
    TS2b = x[9:1280:64]
    TS2c = x[10:1280:64]
    TS2d = x[11:1280:64]
    TS3a = x[12:1280:64]
    TS3b = x[13:1280:64]
    TS3c = x[14:1280:64]
    TS3d = x[15:1280:64]
    TS4a = x[16:1280:64]
    TS4b = x[17:1280:64]
    TS4c = x[18:1280:64]
    TS4d = x[19:1280:64]
    TS5a = x[20:1280:64]
    TS5b = x[21:1280:64]
    TS5c = x[22:1280:64]
    TS5d = x[23:1280:64]
    TS6a = x[24:1280:64]
    TS6b = x[25:1280:64]   
    TS6c = x[26:1280:64]
    TS6d = x[27:1280:64]
    TS7a = x[28:1280:64]
    TS7b = x[29:1280:64]    
    TS7c = x[30:1280:64]
    TS7d = x[31:1280:64]
    TS8a = x[32:1280:64]
    TS8b = x[33:1280:64]
    TS8c = x[34:1280:64]
    TS8d = x[35:1280:64]
    TS9a = x[36:1280:64]
    TS9b = x[37:1280:64]
    TS9c = x[38:1280:64]
    TS9d = x[39:1280:64]
    TS10a = x[40:1280:64]
    TS10b = x[41:1280:64]
    TS10c = x[42:1280:64]
    TS10d = x[43:1280:64]
    TS11a = x[44:1280:64]
    TS11b = x[45:1280:64]
    TS11c = x[46:1280:64]
    TS11d = x[47:1280:64]
    TS12a = x[48:1280:64]
    TS12b = x[49:1280:64]
    TS12c = x[50:1280:64]
    TS12d = x[51:1280:64]
    TS13a = x[52:1280:64]
    TS13b = x[53:1280:64]
    TS13c = x[54:1280:64]
    TS13d = x[55:1280:64]
    TS14a = x[56:1280:64]
    TS14b = x[57:1280:64]
    TS14c = x[58:1280:64]
    TS14d = x[59:1280:64]
    TS15a = x[60:1280:64]
    TS15b = x[61:1280:64]
    TS15c = x[62:1280:64]
    TS15d = x[63:1280:64]
    
    consumption0a = consumption[0:1280:64]
    consumption0b = consumption[1:1280:64]
    consumption0c = consumption[2:1280:64]
    consumption0d = consumption[3:1280:64]
    consumption1a = consumption[4:1280:64]
    consumption1b = consumption[5:1280:64]
    consumption1c = consumption[6:1280:64]
    consumption1d = consumption[7:1280:64]
    consumption2a = consumption[8:1280:64]
    consumption2b = consumption[9:1280:64]
    consumption2c = consumption[10:1280:64]
    consumption2d = consumption[11:1280:64]
    consumption3a = consumption[12:1280:64]
    consumption3b = consumption[13:1280:64]
    consumption3c = consumption[14:1280:64]
    consumption3d = consumption[15:1280:64]
    consumption4a = consumption[16:1280:64]
    consumption4b = consumption[17:1280:64]
    consumption4c = consumption[18:1280:64]
    consumption4d = consumption[19:1280:64]
    consumption5a = consumption[20:1280:64]
    consumption5b = consumption[21:1280:64]
    consumption5c = consumption[22:1280:64]
    consumption5d = consumption[23:1280:64]
    consumption6a = consumption[24:1280:64]
    consumption6b = consumption[25:1280:64]   
    consumption6c = consumption[26:1280:64]
    consumption6d = consumption[27:1280:64]
    consumption7a = consumption[28:1280:64]
    consumption7b = consumption[29:1280:64]    
    consumption7c = consumption[30:1280:64]
    consumption7d = consumption[31:1280:64]
    consumption8a = consumption[32:1280:64]
    consumption8b = consumption[33:1280:64]
    consumption8c = consumption[34:1280:64]
    consumption8d = consumption[35:1280:64]
    consumption9a = consumption[36:1280:64]
    consumption9b = consumption[37:1280:64]
    consumption9c = consumption[38:1280:64]
    consumption9d = consumption[39:1280:64]
    consumption10a = consumption[40:1280:64]
    consumption10b = consumption[41:1280:64]
    consumption10c = consumption[42:1280:64]
    consumption10d = consumption[43:1280:64]
    consumption11a = consumption[44:1280:64]
    consumption11b = consumption[45:1280:64]
    consumption11c = consumption[46:1280:64]
    consumption11d = consumption[47:1280:64]
    consumption12a = consumption[48:1280:64]
    consumption12b = consumption[49:1280:64]
    consumption12c = consumption[50:1280:64]
    consumption12d = consumption[51:1280:64]
    consumption13a = consumption[52:1280:64]
    consumption13b = consumption[53:1280:64]
    consumption13c = consumption[54:1280:64]
    consumption13d = consumption[55:1280:64]
    consumption14a = consumption[56:1280:64]
    consumption14b = consumption[57:1280:64]
    consumption14c = consumption[58:1280:64]
    consumption14d = consumption[59:1280:64]
    consumption15a = consumption[60:1280:64]
    consumption15b = consumption[61:1280:64]
    consumption15c = consumption[62:1280:64]
    consumption15d = consumption[63:1280:64]
    
    #merge arrays
    TS0 = merge4arr(TS0a,TS0b,TS0c,TS0d,80)
    TS1 = merge4arr(TS1a,TS1b,TS1c,TS1d,80)
    TS2 = merge4arr(TS2a,TS2b,TS2c,TS2d,80)
    TS3 = merge4arr(TS3a,TS3b,TS3c,TS3d,80)
    TS4 = merge4arr(TS4a,TS4b,TS4c,TS4d,80)
    TS5 = merge4arr(TS5a,TS5b,TS5c,TS5d,80)
    TS6 = merge4arr(TS6a,TS6b,TS6c,TS6d,80)
    TS7 = merge4arr(TS7a,TS7b,TS7c,TS7d,80)
    TS8 = merge4arr(TS8a,TS8b,TS8c,TS8d,80)
    TS9 = merge4arr(TS9a,TS9b,TS9c,TS9d,80)
    TS10 = merge4arr(TS10a,TS10b,TS10c,TS10d,80)
    TS11 = merge4arr(TS11a,TS11b,TS11c,TS11d,80)
    TS12 = merge4arr(TS12a,TS12b,TS12c,TS12d,80)
    TS13 = merge4arr(TS13a,TS13b,TS13c,TS13d,80)
    TS14 = merge4arr(TS14a,TS14b,TS14c,TS14d,80)
    TS15 = merge4arr(TS15a,TS15b,TS15c,TS15d,80)
	
    consumption0 = merge4arr(consumption0a,consumption0b,consumption0c,consumption0d,80)
    consumption1 = merge4arr(consumption1a,consumption1b,consumption1c,consumption1d,80)
    consumption2 = merge4arr(consumption2a,consumption2b,consumption2c,consumption2d,80)
    consumption3 = merge4arr(consumption3a,consumption3b,consumption3c,consumption3d,80)
    consumption4 = merge4arr(consumption4a,consumption4b,consumption4c,consumption4d,80)
    consumption5 = merge4arr(consumption5a,consumption5b,consumption5c,consumption5d,80)
    consumption6 = merge4arr(consumption6a,consumption6b,consumption6c,consumption6d,80)
    consumption7 = merge4arr(consumption7a,consumption7b,consumption7c,consumption7d,80)
    consumption8 = merge4arr(consumption8a,consumption8b,consumption8c,consumption8d,80)
    consumption9 = merge4arr(consumption9a,consumption9b,consumption9c,consumption9d,80)
    consumption10 = merge4arr(consumption10a,consumption10b,consumption10c,consumption10d,80)
    consumption11 = merge4arr(consumption11a,consumption11b,consumption11c,consumption11d,80)
    consumption12 = merge4arr(consumption12a,consumption12b,consumption12c,consumption12d,80)
    consumption13 = merge4arr(consumption13a,consumption13b,consumption13c,consumption13d,80)
    consumption14 = merge4arr(consumption14a,consumption14b,consumption14c,consumption14d,80)
    consumption15 = merge4arr(consumption15a,consumption15b,consumption15c,consumption15d,80)
    
    #invert array 1 and 0, count 1s to find number of 0s
    inv = np.ones(80, dtype=int)
    
    TS0 = np.subtract(inv,TS0)
    TS1 = np.subtract(inv,TS1)
    TS2 = np.subtract(inv,TS2)
    TS3 = np.subtract(inv,TS3)
    TS4 = np.subtract(inv,TS4)
    TS5 = np.subtract(inv,TS5)
    TS6 = np.subtract(inv,TS6)
    TS7 = np.subtract(inv,TS7)
    TS8 = np.subtract(inv,TS8)
    TS9 = np.subtract(inv,TS9)
    TS10 = np.subtract(inv,TS10)
    TS11 = np.subtract(inv,TS11)
    TS12 = np.subtract(inv,TS12)
    TS13 = np.subtract(inv,TS13)
    TS14 = np.subtract(inv,TS14)
    TS15 = np.subtract(inv,TS15)
    
    #check for instances of not meeting curtailment requirement
    if np.sum(np.multiply(TS0,consumption0)) < TS0limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS1,consumption1)) < TS1limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS2,consumption2)) < TS2limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS3,consumption3)) < TS3limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS4,consumption4)) < TS4limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS5,consumption5)) < TS5limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS6,consumption6)) < TS6limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS7,consumption7)) < TS7limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS8,consumption8)) < TS8limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS9,consumption9)) < TS9limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS10,consumption10)) < TS10limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS11,consumption11)) < TS11limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS12,consumption12)) < TS12limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS13,consumption13)) < TS13limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS14,consumption14)) < TS14limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS15,consumption15)) < TS15limit:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [11]:
def excesscurtail(x,SGE):
    
    #penalty value is for every kW in excess of required curtailment
    penaltyvalue = 9000
    totalpenalty = 0
    
    #set requirement for each TS in kW here
    TS0limit = 110/4
    TS1limit = 220/4
    TS2limit = 455/4
    TS3limit = 680/4
    TS4limit = 770/4
    TS5limit = 800/4
    TS6limit = 750/4
    TS7limit = 640/4
    TS8limit = 590/4
    TS9limit = 610/4
    TS10limit = 660/4
    TS11limit = 680/4
    TS12limit = 570/4
    TS13limit = 410/4
    TS14limit = 230/4
    TS15limit = 135/4
    
    #set consumption for each load in kW here
    ctotal = []
    cons0 = 320/4
    cons1 = 200/4
    cons2 = 80/4
    cons3 = 84/4
    cons4 = 100/4
    cons5 = 160/4
    cons6 = 100/4
    cons7 = 60/4
    cons8 = 200/4
    cons9 = 40/4
    cons10 = 40/4
    cons11 = 72/4
    cons12 = 140/4
    cons13 = 80/4
    cons14 = 40/4
    cons15 = 180/4
    cons16 = 180/4
    cons17 = 160/4
    cons18 = 60/4
    cons19 = 50/4
    
    c0 = np.full(64,cons0)
    c1 = np.full(64,cons1)
    c2 = np.full(64,cons2)
    c3 = np.full(64,cons3)
    c4 = np.full(64,cons4)
    c5 = np.full(64,cons5)
    c6 = np.full(64,cons6)
    c7 = np.full(64,cons7)
    c8 = np.full(64,cons8)
    c9 = np.full(64,cons9)
    c10 = np.full(64,cons10)
    c11 = np.full(64,cons11)
    c12 = np.full(64,cons12)
    c13 = np.full(64,cons13)
    c14 = np.full(64,cons14)
    c15 = np.full(64,cons15)
    c16 = np.full(64,cons16)
    c17 = np.full(64,cons17)
    c18 = np.full(64,cons18)
    c19 = np.full(64,cons19)
    ctotal = np.append(ctotal,[c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19])
    
    #consumption = total req - consumed energy
    consumption = np.subtract(ctotal,SGE)
    
    #cut particles into timeslots here, from TS0 to TS15. Note: 1280 = 20 loads x 64 timeslots
    TS0a = x[0:1280:64]
    TS0b = x[1:1280:64]
    TS0c = x[2:1280:64]
    TS0d = x[3:1280:64]
    TS1a = x[4:1280:64]
    TS1b = x[5:1280:64]
    TS1c = x[6:1280:64]
    TS1d = x[7:1280:64]
    TS2a = x[8:1280:64]
    TS2b = x[9:1280:64]
    TS2c = x[10:1280:64]
    TS2d = x[11:1280:64]
    TS3a = x[12:1280:64]
    TS3b = x[13:1280:64]
    TS3c = x[14:1280:64]
    TS3d = x[15:1280:64]
    TS4a = x[16:1280:64]
    TS4b = x[17:1280:64]
    TS4c = x[18:1280:64]
    TS4d = x[19:1280:64]
    TS5a = x[20:1280:64]
    TS5b = x[21:1280:64]
    TS5c = x[22:1280:64]
    TS5d = x[23:1280:64]
    TS6a = x[24:1280:64]
    TS6b = x[25:1280:64]   
    TS6c = x[26:1280:64]
    TS6d = x[27:1280:64]
    TS7a = x[28:1280:64]
    TS7b = x[29:1280:64]    
    TS7c = x[30:1280:64]
    TS7d = x[31:1280:64]
    TS8a = x[32:1280:64]
    TS8b = x[33:1280:64]
    TS8c = x[34:1280:64]
    TS8d = x[35:1280:64]
    TS9a = x[36:1280:64]
    TS9b = x[37:1280:64]
    TS9c = x[38:1280:64]
    TS9d = x[39:1280:64]
    TS10a = x[40:1280:64]
    TS10b = x[41:1280:64]
    TS10c = x[42:1280:64]
    TS10d = x[43:1280:64]
    TS11a = x[44:1280:64]
    TS11b = x[45:1280:64]
    TS11c = x[46:1280:64]
    TS11d = x[47:1280:64]
    TS12a = x[48:1280:64]
    TS12b = x[49:1280:64]
    TS12c = x[50:1280:64]
    TS12d = x[51:1280:64]
    TS13a = x[52:1280:64]
    TS13b = x[53:1280:64]
    TS13c = x[54:1280:64]
    TS13d = x[55:1280:64]
    TS14a = x[56:1280:64]
    TS14b = x[57:1280:64]
    TS14c = x[58:1280:64]
    TS14d = x[59:1280:64]
    TS15a = x[60:1280:64]
    TS15b = x[61:1280:64]
    TS15c = x[62:1280:64]
    TS15d = x[63:1280:64]
    
    consumption0a = consumption[0:1280:64]
    consumption0b = consumption[1:1280:64]
    consumption0c = consumption[2:1280:64]
    consumption0d = consumption[3:1280:64]
    consumption1a = consumption[4:1280:64]
    consumption1b = consumption[5:1280:64]
    consumption1c = consumption[6:1280:64]
    consumption1d = consumption[7:1280:64]
    consumption2a = consumption[8:1280:64]
    consumption2b = consumption[9:1280:64]
    consumption2c = consumption[10:1280:64]
    consumption2d = consumption[11:1280:64]
    consumption3a = consumption[12:1280:64]
    consumption3b = consumption[13:1280:64]
    consumption3c = consumption[14:1280:64]
    consumption3d = consumption[15:1280:64]
    consumption4a = consumption[16:1280:64]
    consumption4b = consumption[17:1280:64]
    consumption4c = consumption[18:1280:64]
    consumption4d = consumption[19:1280:64]
    consumption5a = consumption[20:1280:64]
    consumption5b = consumption[21:1280:64]
    consumption5c = consumption[22:1280:64]
    consumption5d = consumption[23:1280:64]
    consumption6a = consumption[24:1280:64]
    consumption6b = consumption[25:1280:64]   
    consumption6c = consumption[26:1280:64]
    consumption6d = consumption[27:1280:64]
    consumption7a = consumption[28:1280:64]
    consumption7b = consumption[29:1280:64]    
    consumption7c = consumption[30:1280:64]
    consumption7d = consumption[31:1280:64]
    consumption8a = consumption[32:1280:64]
    consumption8b = consumption[33:1280:64]
    consumption8c = consumption[34:1280:64]
    consumption8d = consumption[35:1280:64]
    consumption9a = consumption[36:1280:64]
    consumption9b = consumption[37:1280:64]
    consumption9c = consumption[38:1280:64]
    consumption9d = consumption[39:1280:64]
    consumption10a = consumption[40:1280:64]
    consumption10b = consumption[41:1280:64]
    consumption10c = consumption[42:1280:64]
    consumption10d = consumption[43:1280:64]
    consumption11a = consumption[44:1280:64]
    consumption11b = consumption[45:1280:64]
    consumption11c = consumption[46:1280:64]
    consumption11d = consumption[47:1280:64]
    consumption12a = consumption[48:1280:64]
    consumption12b = consumption[49:1280:64]
    consumption12c = consumption[50:1280:64]
    consumption12d = consumption[51:1280:64]
    consumption13a = consumption[52:1280:64]
    consumption13b = consumption[53:1280:64]
    consumption13c = consumption[54:1280:64]
    consumption13d = consumption[55:1280:64]
    consumption14a = consumption[56:1280:64]
    consumption14b = consumption[57:1280:64]
    consumption14c = consumption[58:1280:64]
    consumption14d = consumption[59:1280:64]
    consumption15a = consumption[60:1280:64]
    consumption15b = consumption[61:1280:64]
    consumption15c = consumption[62:1280:64]
    consumption15d = consumption[63:1280:64]
    
    #merge arrays
    TS0 = merge4arr(TS0a,TS0b,TS0c,TS0d,80)
    TS1 = merge4arr(TS1a,TS1b,TS1c,TS1d,80)
    TS2 = merge4arr(TS2a,TS2b,TS2c,TS2d,80)
    TS3 = merge4arr(TS3a,TS3b,TS3c,TS3d,80)
    TS4 = merge4arr(TS4a,TS4b,TS4c,TS4d,80)
    TS5 = merge4arr(TS5a,TS5b,TS5c,TS5d,80)
    TS6 = merge4arr(TS6a,TS6b,TS6c,TS6d,80)
    TS7 = merge4arr(TS7a,TS7b,TS7c,TS7d,80)
    TS8 = merge4arr(TS8a,TS8b,TS8c,TS8d,80)
    TS9 = merge4arr(TS9a,TS9b,TS9c,TS9d,80)
    TS10 = merge4arr(TS10a,TS10b,TS10c,TS10d,80)
    TS11 = merge4arr(TS11a,TS11b,TS11c,TS11d,80)
    TS12 = merge4arr(TS12a,TS12b,TS12c,TS12d,80)
    TS13 = merge4arr(TS13a,TS13b,TS13c,TS13d,80)
    TS14 = merge4arr(TS14a,TS14b,TS14c,TS14d,80)
    TS15 = merge4arr(TS15a,TS15b,TS15c,TS15d,80)
    consumption0 = merge4arr(consumption0a,consumption0b,consumption0c,consumption0d,80)
    consumption1 = merge4arr(consumption1a,consumption1b,consumption1c,consumption1d,80)
    consumption2 = merge4arr(consumption2a,consumption2b,consumption2c,consumption2d,80)
    consumption3 = merge4arr(consumption3a,consumption3b,consumption3c,consumption3d,80)
    consumption4 = merge4arr(consumption4a,consumption4b,consumption4c,consumption4d,80)
    consumption5 = merge4arr(consumption5a,consumption5b,consumption5c,consumption5d,80)
    consumption6 = merge4arr(consumption6a,consumption6b,consumption6c,consumption6d,80)
    consumption7 = merge4arr(consumption7a,consumption7b,consumption7c,consumption7d,80)
    consumption8 = merge4arr(consumption8a,consumption8b,consumption8c,consumption8d,80)
    consumption9 = merge4arr(consumption9a,consumption9b,consumption9c,consumption9d,80)
    consumption10 = merge4arr(consumption10a,consumption10b,consumption10c,consumption10d,80)
    consumption11 = merge4arr(consumption11a,consumption11b,consumption11c,consumption11d,80)
    consumption12 = merge4arr(consumption12a,consumption12b,consumption12c,consumption12d,80)
    consumption13 = merge4arr(consumption13a,consumption13b,consumption13c,consumption13d,80)
    consumption14 = merge4arr(consumption14a,consumption14b,consumption14c,consumption14d,80)
    consumption15 = merge4arr(consumption15a,consumption15b,consumption15c,consumption15d,80)
    
    #invert array 1 and 0, count 1s to find number of 0s
    inv = np.ones(80, dtype=int)
    
    TS0 = np.subtract(inv,TS0)
    TS1 = np.subtract(inv,TS1)
    TS2 = np.subtract(inv,TS2)
    TS3 = np.subtract(inv,TS3)
    TS4 = np.subtract(inv,TS4)
    TS5 = np.subtract(inv,TS5)
    TS6 = np.subtract(inv,TS6)
    TS7 = np.subtract(inv,TS7)
    TS8 = np.subtract(inv,TS8)
    TS9 = np.subtract(inv,TS9)
    TS10 = np.subtract(inv,TS10)
    TS11 = np.subtract(inv,TS11)
    TS12 = np.subtract(inv,TS12)
    TS13 = np.subtract(inv,TS13)
    TS14 = np.subtract(inv,TS14)
    TS15 = np.subtract(inv,TS15)
    
    excess0 = np.sum(np.multiply(TS0,consumption0)) - TS0limit
    excess1 = np.sum(np.multiply(TS1,consumption1)) - TS1limit
    excess2 = np.sum(np.multiply(TS2,consumption2)) - TS2limit
    excess3 = np.sum(np.multiply(TS3,consumption3)) - TS3limit
    excess4 = np.sum(np.multiply(TS4,consumption4)) - TS4limit
    excess5 = np.sum(np.multiply(TS5,consumption5)) - TS5limit
    excess6 = np.sum(np.multiply(TS6,consumption6)) - TS6limit
    excess7 = np.sum(np.multiply(TS7,consumption7)) - TS7limit
    excess8 = np.sum(np.multiply(TS8,consumption8)) - TS8limit
    excess9 = np.sum(np.multiply(TS9,consumption9)) - TS9limit
    excess10 = np.sum(np.multiply(TS10,consumption10)) - TS10limit
    excess11 = np.sum(np.multiply(TS11,consumption11)) - TS11limit
    excess12 = np.sum(np.multiply(TS12,consumption12)) - TS12limit
    excess13 = np.sum(np.multiply(TS13,consumption13)) - TS13limit
    excess14 = np.sum(np.multiply(TS14,consumption14)) - TS14limit
    excess15 = np.sum(np.multiply(TS15,consumption15)) - TS15limit
    
    excess0 = abs(excess0)
    excess1 = abs(excess1)
    excess2 = abs(excess2)
    excess3 = abs(excess3)
    excess4 = abs(excess4)
    excess5 = abs(excess5)
    excess6 = abs(excess6)
    excess7 = abs(excess7)
    excess8 = abs(excess8)
    excess9 = abs(excess9)
    excess10 = abs(excess10)
    excess11 = abs(excess11)
    excess12 = abs(excess12)
    excess13 = abs(excess13)
    excess14 = abs(excess14)
    excess15 = abs(excess15)
    
    totalexcess = excess0 + excess1 + excess2 + excess3 + excess4 + excess5 + excess6 + excess7 + excess8 + excess9 +\
    excess10 + excess11 + excess12 + excess13 + excess14 + excess15
    
    totalpenalty = totalexcess*penaltyvalue
    
    return totalpenalty

In [12]:
#function to make sure the curtailments are in continuous chunks, ideally one long curtailment
def continuity(x):
    #timeslotrange = 1280 elements per particle divided by 20 loads
    timeslotrange = 64
    penaltyvalue = 10000
    totalpenalty = 0
    
    #divide array into 20 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    
    #if there are more than 1 transitions from 1 to 0, not one long interrupt, add penalty 
    
    if np.count_nonzero(np.diff(L0)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L1)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L2)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L3)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L4)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L5)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L6)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L7)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L8)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L9)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L10)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L11)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L12)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L13)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L14)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L15)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L16)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L17)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L18)==-1) > 1:
        totalpenalty += 1
    if np.count_nonzero(np.diff(L19)==-1) > 1:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [13]:
#test function, 2D version, to be implemented per element (Element is 1D array)
def obj_func(x,SGE):
    
    #timeslotrange = 1280 elements per particle divided by 20 loads
    timeslotrange = 64
    
    #set costs per kW here
    DUcost = 10/4
    SGEcost = 5/4
    
    #set consumption for each load in kW here
    ctotal = []
    cons0 = 320/4
    cons1 = 200/4
    cons2 = 80/4
    cons3 = 84/4
    cons4 = 100/4
    cons5 = 160/4
    cons6 = 100/4
    cons7 = 60/4
    cons8 = 200/4
    cons9 = 40/4
    cons10 = 40/4
    cons11 = 72/4
    cons12 = 140/4
    cons13 = 80/4
    cons14 = 40/4
    cons15 = 180/4
    cons16 = 180/4
    cons17 = 160/4
    cons18 = 60/4
    cons19 = 50/4
    
    c0 = np.full(64,cons0)
    c1 = np.full(64,cons1)
    c2 = np.full(64,cons2)
    c3 = np.full(64,cons3)
    c4 = np.full(64,cons4)
    c5 = np.full(64,cons5)
    c6 = np.full(64,cons6)
    c7 = np.full(64,cons7)
    c8 = np.full(64,cons8)
    c9 = np.full(64,cons9)
    c10 = np.full(64,cons10)
    c11 = np.full(64,cons11)
    c12 = np.full(64,cons12)
    c13 = np.full(64,cons13)
    c14 = np.full(64,cons14)
    c15 = np.full(64,cons15)
    c16 = np.full(64,cons16)
    c17 = np.full(64,cons17)
    c18 = np.full(64,cons18)
    c19 = np.full(64,cons19)
    
    #self generated energy, divide into 20 loads x 64 timeslots
    SGE0 = SGE[0:timeslotrange]
    SGE1 = SGE[timeslotrange:(timeslotrange*2)]
    SGE2 = SGE[(timeslotrange*2):(timeslotrange*3)]
    SGE3 = SGE[(timeslotrange*3):(timeslotrange*4)]
    SGE4 = SGE[(timeslotrange*4):(timeslotrange*5)]
    SGE5 = SGE[(timeslotrange*5):(timeslotrange*6)]
    SGE6 = SGE[(timeslotrange*6):(timeslotrange*7)]
    SGE7 = SGE[(timeslotrange*7):(timeslotrange*8)]
    SGE8 = SGE[(timeslotrange*8):(timeslotrange*9)]
    SGE9 = SGE[(timeslotrange*9):(timeslotrange*10)]
    SGE10 = SGE[(timeslotrange*10):(timeslotrange*11)]
    SGE11 = SGE[(timeslotrange*11):(timeslotrange*12)]
    SGE12 = SGE[(timeslotrange*12):(timeslotrange*13)]
    SGE13 = SGE[(timeslotrange*13):(timeslotrange*14)]
    SGE14 = SGE[(timeslotrange*14):(timeslotrange*15)]
    SGE15 = SGE[(timeslotrange*15):(timeslotrange*16)]
    SGE16 = SGE[(timeslotrange*16):(timeslotrange*17)]
    SGE17 = SGE[(timeslotrange*17):(timeslotrange*18)]
    SGE18 = SGE[(timeslotrange*18):(timeslotrange*19)]
    SGE19 = SGE[(timeslotrange*19):(timeslotrange*20)]

    #divide array into 20 loads x 64 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    #sum of all the loads, each load is [schedule x DUcost x (Consumption-SGE)] + [schedule x SGEcost x SGE]
    
    f = np.sum(np.multiply(np.multiply(np.subtract(c0,SGE0),DUcost),np.array(L0))) + np.sum(np.multiply(np.multiply(SGE0,SGEcost),np.array(L0))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c1,SGE1),DUcost),np.array(L1))) + np.sum(np.multiply(np.multiply(SGE1,SGEcost),np.array(L1))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c2,SGE2),DUcost),np.array(L2))) + np.sum(np.multiply(np.multiply(SGE2,SGEcost),np.array(L2))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c3,SGE3),DUcost),np.array(L3))) + np.sum(np.multiply(np.multiply(SGE3,SGEcost),np.array(L3))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c4,SGE4),DUcost),np.array(L4))) + np.sum(np.multiply(np.multiply(SGE4,SGEcost),np.array(L4))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c5,SGE5),DUcost),np.array(L5))) + np.sum(np.multiply(np.multiply(SGE5,SGEcost),np.array(L5))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c6,SGE6),DUcost),np.array(L6))) + np.sum(np.multiply(np.multiply(SGE6,SGEcost),np.array(L6))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c7,SGE7),DUcost),np.array(L7))) + np.sum(np.multiply(np.multiply(SGE7,SGEcost),np.array(L7))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c8,SGE8),DUcost),np.array(L8))) + np.sum(np.multiply(np.multiply(SGE8,SGEcost),np.array(L8))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c9,SGE9),DUcost),np.array(L9))) + np.sum(np.multiply(np.multiply(SGE9,SGEcost),np.array(L9))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c10,SGE10),DUcost),np.array(L10))) + np.sum(np.multiply(np.multiply(SGE10,SGEcost),np.array(L10))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c11,SGE11),DUcost),np.array(L11))) + np.sum(np.multiply(np.multiply(SGE11,SGEcost),np.array(L11))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c12,SGE12),DUcost),np.array(L12))) + np.sum(np.multiply(np.multiply(SGE12,SGEcost),np.array(L12))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c13,SGE13),DUcost),np.array(L13))) + np.sum(np.multiply(np.multiply(SGE13,SGEcost),np.array(L13))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c14,SGE14),DUcost),np.array(L14))) + np.sum(np.multiply(np.multiply(SGE14,SGEcost),np.array(L14))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c15,SGE15),DUcost),np.array(L15))) + np.sum(np.multiply(np.multiply(SGE15,SGEcost),np.array(L15))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c16,SGE16),DUcost),np.array(L16))) + np.sum(np.multiply(np.multiply(SGE16,SGEcost),np.array(L16))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c17,SGE17),DUcost),np.array(L17))) + np.sum(np.multiply(np.multiply(SGE17,SGEcost),np.array(L17))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c18,SGE18),DUcost),np.array(L18))) + np.sum(np.multiply(np.multiply(SGE18,SGEcost),np.array(L18))) +\
    np.sum(np.multiply(np.multiply(np.subtract(c19,SGE19),DUcost),np.array(L19))) + np.sum(np.multiply(np.multiply(SGE19,SGEcost),np.array(L19))) +\
    overcurtail(x) + undercurtail(x) + reqcurtail(x,SGE) + excesscurtail(x,SGE) + continuity(x)
    
    return f


In [14]:
def checkvalidity(x,SGE):
     #set requirement for each TS in kW here
    TS0limit = 110/4
    TS1limit = 220/4
    TS2limit = 455/4
    TS3limit = 680/4
    TS4limit = 770/4
    TS5limit = 800/4
    TS6limit = 750/4
    TS7limit = 640/4
    TS8limit = 590/4
    TS9limit = 610/4
    TS10limit = 660/4
    TS11limit = 680/4
    TS12limit = 570/4
    TS13limit = 410/4
    TS14limit = 230/4
    TS15limit = 135/4
    
    #set consumption for each load in kW here
    ctotal = []
    cons0 = 320/4
    cons1 = 200/4
    cons2 = 80/4
    cons3 = 84/4
    cons4 = 100/4
    cons5 = 160/4
    cons6 = 100/4
    cons7 = 60/4
    cons8 = 200/4
    cons9 = 40/4
    cons10 = 40/4
    cons11 = 72/4
    cons12 = 140/4
    cons13 = 80/4
    cons14 = 40/4
    cons15 = 180/4
    cons16 = 180/4
    cons17 = 160/4
    cons18 = 60/4
    cons19 = 50/4
    
    c0 = np.full(64,cons0)
    c1 = np.full(64,cons1)
    c2 = np.full(64,cons2)
    c3 = np.full(64,cons3)
    c4 = np.full(64,cons4)
    c5 = np.full(64,cons5)
    c6 = np.full(64,cons6)
    c7 = np.full(64,cons7)
    c8 = np.full(64,cons8)
    c9 = np.full(64,cons9)
    c10 = np.full(64,cons10)
    c11 = np.full(64,cons11)
    c12 = np.full(64,cons12)
    c13 = np.full(64,cons13)
    c14 = np.full(64,cons14)
    c15 = np.full(64,cons15)
    c16 = np.full(64,cons16)
    c17 = np.full(64,cons17)
    c18 = np.full(64,cons18)
    c19 = np.full(64,cons19)
    ctotal = np.append(ctotal,[c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19])
    
    #consumption = total req - consumed energy
    consumption = np.subtract(ctotal,SGE)
   
    #cut particles into timeslots here, from TS0 to TS15. Note: 1280 = 20 loads x 64 timeslots
    TS0a = x[0:1280:64]
    TS0b = x[1:1280:64]
    TS0c = x[2:1280:64]
    TS0d = x[3:1280:64]
    TS1a = x[4:1280:64]
    TS1b = x[5:1280:64]
    TS1c = x[6:1280:64]
    TS1d = x[7:1280:64]
    TS2a = x[8:1280:64]
    TS2b = x[9:1280:64]
    TS2c = x[10:1280:64]
    TS2d = x[11:1280:64]
    TS3a = x[12:1280:64]
    TS3b = x[13:1280:64]
    TS3c = x[14:1280:64]
    TS3d = x[15:1280:64]
    TS4a = x[16:1280:64]
    TS4b = x[17:1280:64]
    TS4c = x[18:1280:64]
    TS4d = x[19:1280:64]
    TS5a = x[20:1280:64]
    TS5b = x[21:1280:64]
    TS5c = x[22:1280:64]
    TS5d = x[23:1280:64]
    TS6a = x[24:1280:64]
    TS6b = x[25:1280:64]   
    TS6c = x[26:1280:64]
    TS6d = x[27:1280:64]
    TS7a = x[28:1280:64]
    TS7b = x[29:1280:64]    
    TS7c = x[30:1280:64]
    TS7d = x[31:1280:64]
    TS8a = x[32:1280:64]
    TS8b = x[33:1280:64]
    TS8c = x[34:1280:64]
    TS8d = x[35:1280:64]
    TS9a = x[36:1280:64]
    TS9b = x[37:1280:64]
    TS9c = x[38:1280:64]
    TS9d = x[39:1280:64]
    TS10a = x[40:1280:64]
    TS10b = x[41:1280:64]
    TS10c = x[42:1280:64]
    TS10d = x[43:1280:64]
    TS11a = x[44:1280:64]
    TS11b = x[45:1280:64]
    TS11c = x[46:1280:64]
    TS11d = x[47:1280:64]
    TS12a = x[48:1280:64]
    TS12b = x[49:1280:64]
    TS12c = x[50:1280:64]
    TS12d = x[51:1280:64]
    TS13a = x[52:1280:64]
    TS13b = x[53:1280:64]
    TS13c = x[54:1280:64]
    TS13d = x[55:1280:64]
    TS14a = x[56:1280:64]
    TS14b = x[57:1280:64]
    TS14c = x[58:1280:64]
    TS14d = x[59:1280:64]
    TS15a = x[60:1280:64]
    TS15b = x[61:1280:64]
    TS15c = x[62:1280:64]
    TS15d = x[63:1280:64]
    
    consumption0a = consumption[0:1280:64]
    consumption0b = consumption[1:1280:64]
    consumption0c = consumption[2:1280:64]
    consumption0d = consumption[3:1280:64]
    consumption1a = consumption[4:1280:64]
    consumption1b = consumption[5:1280:64]
    consumption1c = consumption[6:1280:64]
    consumption1d = consumption[7:1280:64]
    consumption2a = consumption[8:1280:64]
    consumption2b = consumption[9:1280:64]
    consumption2c = consumption[10:1280:64]
    consumption2d = consumption[11:1280:64]
    consumption3a = consumption[12:1280:64]
    consumption3b = consumption[13:1280:64]
    consumption3c = consumption[14:1280:64]
    consumption3d = consumption[15:1280:64]
    consumption4a = consumption[16:1280:64]
    consumption4b = consumption[17:1280:64]
    consumption4c = consumption[18:1280:64]
    consumption4d = consumption[19:1280:64]
    consumption5a = consumption[20:1280:64]
    consumption5b = consumption[21:1280:64]
    consumption5c = consumption[22:1280:64]
    consumption5d = consumption[23:1280:64]
    consumption6a = consumption[24:1280:64]
    consumption6b = consumption[25:1280:64]   
    consumption6c = consumption[26:1280:64]
    consumption6d = consumption[27:1280:64]
    consumption7a = consumption[28:1280:64]
    consumption7b = consumption[29:1280:64]    
    consumption7c = consumption[30:1280:64]
    consumption7d = consumption[31:1280:64]
    consumption8a = consumption[32:1280:64]
    consumption8b = consumption[33:1280:64]
    consumption8c = consumption[34:1280:64]
    consumption8d = consumption[35:1280:64]
    consumption9a = consumption[36:1280:64]
    consumption9b = consumption[37:1280:64]
    consumption9c = consumption[38:1280:64]
    consumption9d = consumption[39:1280:64]
    consumption10a = consumption[40:1280:64]
    consumption10b = consumption[41:1280:64]
    consumption10c = consumption[42:1280:64]
    consumption10d = consumption[43:1280:64]
    consumption11a = consumption[44:1280:64]
    consumption11b = consumption[45:1280:64]
    consumption11c = consumption[46:1280:64]
    consumption11d = consumption[47:1280:64]
    consumption12a = consumption[48:1280:64]
    consumption12b = consumption[49:1280:64]
    consumption12c = consumption[50:1280:64]
    consumption12d = consumption[51:1280:64]
    consumption13a = consumption[52:1280:64]
    consumption13b = consumption[53:1280:64]
    consumption13c = consumption[54:1280:64]
    consumption13d = consumption[55:1280:64]
    consumption14a = consumption[56:1280:64]
    consumption14b = consumption[57:1280:64]
    consumption14c = consumption[58:1280:64]
    consumption14d = consumption[59:1280:64]
    consumption15a = consumption[60:1280:64]
    consumption15b = consumption[61:1280:64]
    consumption15c = consumption[62:1280:64]
    consumption15d = consumption[63:1280:64]
    
    #merge arrays
    TS0 = merge4arr(TS0a,TS0b,TS0c,TS0d,80)
    TS1 = merge4arr(TS1a,TS1b,TS1c,TS1d,80)
    TS2 = merge4arr(TS2a,TS2b,TS2c,TS2d,80)
    TS3 = merge4arr(TS3a,TS3b,TS3c,TS3d,80)
    TS4 = merge4arr(TS4a,TS4b,TS4c,TS4d,80)
    TS5 = merge4arr(TS5a,TS5b,TS5c,TS5d,80)
    TS6 = merge4arr(TS6a,TS6b,TS6c,TS6d,80)
    TS7 = merge4arr(TS7a,TS7b,TS7c,TS7d,80)
    TS8 = merge4arr(TS8a,TS8b,TS8c,TS8d,80)
    TS9 = merge4arr(TS9a,TS9b,TS9c,TS9d,80)
    TS10 = merge4arr(TS10a,TS10b,TS10c,TS10d,80)
    TS11 = merge4arr(TS11a,TS11b,TS11c,TS11d,80)
    TS12 = merge4arr(TS12a,TS12b,TS12c,TS12d,80)
    TS13 = merge4arr(TS13a,TS13b,TS13c,TS13d,80)
    TS14 = merge4arr(TS14a,TS14b,TS14c,TS14d,80)
    TS15 = merge4arr(TS15a,TS15b,TS15c,TS15d,80)
    consumption0 = merge4arr(consumption0a,consumption0b,consumption0c,consumption0d,80)
    consumption1 = merge4arr(consumption1a,consumption1b,consumption1c,consumption1d,80)
    consumption2 = merge4arr(consumption2a,consumption2b,consumption2c,consumption2d,80)
    consumption3 = merge4arr(consumption3a,consumption3b,consumption3c,consumption3d,80)
    consumption4 = merge4arr(consumption4a,consumption4b,consumption4c,consumption4d,80)
    consumption5 = merge4arr(consumption5a,consumption5b,consumption5c,consumption5d,80)
    consumption6 = merge4arr(consumption6a,consumption6b,consumption6c,consumption6d,80)
    consumption7 = merge4arr(consumption7a,consumption7b,consumption7c,consumption7d,80)
    consumption8 = merge4arr(consumption8a,consumption8b,consumption8c,consumption8d,80)
    consumption9 = merge4arr(consumption9a,consumption9b,consumption9c,consumption9d,80)
    consumption10 = merge4arr(consumption10a,consumption10b,consumption10c,consumption10d,80)
    consumption11 = merge4arr(consumption11a,consumption11b,consumption11c,consumption11d,80)
    consumption12 = merge4arr(consumption12a,consumption12b,consumption12c,consumption12d,80)
    consumption13 = merge4arr(consumption13a,consumption13b,consumption13c,consumption13d,80)
    consumption14 = merge4arr(consumption14a,consumption14b,consumption14c,consumption14d,80)
    consumption15 = merge4arr(consumption15a,consumption15b,consumption15c,consumption15d,80)
    
    #invert array 1 and 0, count 1s to find number of 0s
    inv = np.ones(80, dtype=int)
    
    TS0 = np.subtract(inv,TS0)
    TS1 = np.subtract(inv,TS1)
    TS2 = np.subtract(inv,TS2)
    TS3 = np.subtract(inv,TS3)
    TS4 = np.subtract(inv,TS4)
    TS5 = np.subtract(inv,TS5)
    TS6 = np.subtract(inv,TS6)
    TS7 = np.subtract(inv,TS7)
    TS8 = np.subtract(inv,TS8)
    TS9 = np.subtract(inv,TS9)
    TS10 = np.subtract(inv,TS10)
    TS11 = np.subtract(inv,TS11)
    TS12 = np.subtract(inv,TS12)
    TS13 = np.subtract(inv,TS13)
    TS14 = np.subtract(inv,TS14)
    TS15 = np.subtract(inv,TS15)
    
    excess0 = np.sum(np.multiply(TS0,consumption0)) - TS0limit
    excess1 = np.sum(np.multiply(TS1,consumption1)) - TS1limit
    excess2 = np.sum(np.multiply(TS2,consumption2)) - TS2limit
    excess3 = np.sum(np.multiply(TS3,consumption3)) - TS3limit
    excess4 = np.sum(np.multiply(TS4,consumption4)) - TS4limit
    excess5 = np.sum(np.multiply(TS5,consumption5)) - TS5limit
    excess6 = np.sum(np.multiply(TS6,consumption6)) - TS6limit
    excess7 = np.sum(np.multiply(TS7,consumption7)) - TS7limit
    excess8 = np.sum(np.multiply(TS8,consumption8)) - TS8limit
    excess9 = np.sum(np.multiply(TS9,consumption9)) - TS9limit
    excess10 = np.sum(np.multiply(TS10,consumption10)) - TS10limit
    excess11 = np.sum(np.multiply(TS11,consumption11)) - TS11limit
    excess12 = np.sum(np.multiply(TS12,consumption12)) - TS12limit
    excess13 = np.sum(np.multiply(TS13,consumption13)) - TS13limit
    excess14 = np.sum(np.multiply(TS14,consumption14)) - TS14limit
    excess15 = np.sum(np.multiply(TS15,consumption15)) - TS15limit
    
    excessarray = [excess0, excess1, excess2, excess3, excess4, excess5, excess6, excess7, excess8, excess9, excess10, excess11, excess12, excess13, excess14, excess15]
    print("Array of overcurtailment in kW:\n", excessarray)
    print("Total overcurtailment across all timeslots:\n",np.sum(excessarray))
    print("\n")

    #parse to loads
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    L10 = x[(timeslotrange*10):(timeslotrange*11)]
    L11 = x[(timeslotrange*11):(timeslotrange*12)]
    L12 = x[(timeslotrange*12):(timeslotrange*13)]
    L13 = x[(timeslotrange*13):(timeslotrange*14)]
    L14 = x[(timeslotrange*14):(timeslotrange*15)]
    L15 = x[(timeslotrange*15):(timeslotrange*16)]
    L16 = x[(timeslotrange*16):(timeslotrange*17)]
    L17 = x[(timeslotrange*17):(timeslotrange*18)]
    L18 = x[(timeslotrange*18):(timeslotrange*19)]
    L19 = x[(timeslotrange*19):(timeslotrange*20)]
    
    #minimum on time
    minonL0 = 2*4
    minonL1 = 2*4
    minonL2 = 2*4
    minonL3 = 2*4
    minonL4 = 2*4
    minonL5 = 2*4
    minonL6 = 1*4
    minonL7 = 1*4
    minonL8 = 1*4
    minonL9 = 1*4
    minonL10 = 1*4
    minonL11 = 3*4
    minonL12 = 3*4
    minonL13 = 3*4
    minonL14 = 3*4
    minonL15 = 3*4
    minonL16 = 2*4
    minonL17 = 2*4
    minonL18 = 2*4
    minonL19 = 2*4
    
    #maximum off time
    maxoffL0 = 4*4
    maxoffL1 = 4*4
    maxoffL2 = 4*4
    maxoffL3 = 4*4
    maxoffL4 = 4*4
    maxoffL5 = 4*4
    maxoffL6 = 3*4
    maxoffL7 = 3*4
    maxoffL8 = 3*4
    maxoffL9 = 4*4
    maxoffL10 = 3*4
    maxoffL11 = 3*4
    maxoffL12 = 4*4
    maxoffL13 = 3*4
    maxoffL14 = 3*4
    maxoffL15 = 3*4
    maxoffL16 = 4*4
    maxoffL17 = 4*4
    maxoffL18 = 4*4
    maxoffL19 = 4*4
    
    #printing values
    ontimeL0 = np.count_nonzero(L0==1)
    offtimeL0 = 64 - ontimeL0
    print("L0 min on time:", minonL0)
    print("L0 on time:", ontimeL0)
    print("L0 max off time:", maxoffL0)
    print("L0 off time:", offtimeL0)
    print("\n")
    
    ontimeL1 = np.count_nonzero(L1==1)
    offtimeL1 = 64 - ontimeL1
    print("L1 min on time:", minonL1)
    print("L1 on time:", ontimeL1)
    print("L1 max off time:", maxoffL1)
    print("L1 off time:", offtimeL1)
    print("\n")
    
    ontimeL2 = np.count_nonzero(L2==1)
    offtimeL2 = 64 - ontimeL2
    print("L2 min on time:", minonL2)
    print("L2 on time:", ontimeL2)
    print("L2 max off time:", maxoffL2)
    print("L2 off time:", offtimeL2)
    print("\n")
    
    ontimeL3 = np.count_nonzero(L3==1)
    offtimeL3 = 64 - ontimeL3
    print("L3 min on time:", minonL3)
    print("L3 on time:", ontimeL3)
    print("L3 max off time:", maxoffL3)
    print("L3 off time:", offtimeL3)
    print("\n")
    
    ontimeL4 = np.count_nonzero(L4==1)
    offtimeL4 = 64 - ontimeL4
    print("L4 min on time:", minonL4)
    print("L4 on time:", ontimeL4)
    print("L4 max off time:", maxoffL4)
    print("L4 off time:", offtimeL4)
    print("\n")
    
    ontimeL5 = np.count_nonzero(L5==1)
    offtimeL5 = 64 - ontimeL5
    print("L5 min on time:", minonL5)
    print("L5 on time:", ontimeL5)
    print("L5 max off time:", maxoffL5)
    print("L5 off time:", offtimeL5)
    print("\n")
    
    ontimeL6 = np.count_nonzero(L6==1)
    offtimeL6 = 64 - ontimeL6
    print("L6 min on time:", minonL6)
    print("L6 on time:", ontimeL6)
    print("L6 max off time:", maxoffL6)
    print("L6 off time:", offtimeL6)
    print("\n")
    
    ontimeL7 = np.count_nonzero(L7==1)
    offtimeL7 = 64 - ontimeL7
    print("L7 min on time:", minonL7)
    print("L7 on time:", ontimeL7)
    print("L7 max off time:", maxoffL7)
    print("L7 off time:", offtimeL7)
    print("\n")
    
    ontimeL8 = np.count_nonzero(L8==1)
    offtimeL8 = 64 - ontimeL8
    print("L8 min on time:", minonL8)
    print("L8 on time:", ontimeL8)
    print("L8 max off time:", maxoffL8)
    print("L8 off time:", offtimeL8)
    print("\n")
    
    ontimeL9 = np.count_nonzero(L9==1)
    offtimeL9 = 64 - ontimeL9
    print("L9 min on time:", minonL9)
    print("L9 on time:", ontimeL9)
    print("L9 max off time:", maxoffL9)
    print("L9 off time:", offtimeL9)
    print("\n")
    
    ontimeL10 = np.count_nonzero(L10==1)
    offtimeL10 = 64 - ontimeL10
    print("L10 min on time:", minonL10)
    print("L10 on time:", ontimeL10)
    print("L10 max off time:", maxoffL10)
    print("L10 off time:", offtimeL10)
    print("\n")
    
    ontimeL11 = np.count_nonzero(L11==1)
    offtimeL11 = 64 - ontimeL11
    print("L11 min on time:", minonL11)
    print("L11 on time:", ontimeL11)
    print("L11 max off time:", maxoffL11)
    print("L11 off time:", offtimeL11)
    print("\n")
    
    ontimeL12 = np.count_nonzero(L12==1)
    offtimeL12 = 64 - ontimeL12
    print("L12 min on time:", minonL12)
    print("L12 on time:", ontimeL12)
    print("L12 max off time:", maxoffL12)
    print("L12 off time:", offtimeL12)
    print("\n")
    
    ontimeL13 = np.count_nonzero(L13==1)
    offtimeL13 = 64 - ontimeL13
    print("L13 min on time:", minonL13)
    print("L13 on time:", ontimeL13)
    print("L13 max off time:", maxoffL13)
    print("L13 off time:", offtimeL13)
    print("\n")
    
    ontimeL14 = np.count_nonzero(L14==1)
    offtimeL14 = 64 - ontimeL14
    print("L14 min on time:", minonL14)
    print("L14 on time:", ontimeL14)
    print("L14 max off time:", maxoffL14)
    print("L14 off time:", offtimeL14)
    print("\n")
    
    ontimeL15 = np.count_nonzero(L15==1)
    offtimeL15 = 64 - ontimeL15
    print("L15 min on time:", minonL15)
    print("L15 on time:", ontimeL15)
    print("L15 max off time:", maxoffL15)
    print("L15 off time:", offtimeL15)
    print("\n")
    
    ontimeL16 = np.count_nonzero(L16==1)
    offtimeL16 = 64 - ontimeL16
    print("L16 min on time:", minonL16)
    print("L16 on time:", ontimeL16)
    print("L16 max off time:", maxoffL16)
    print("L16 off time:", offtimeL16)
    print("\n")
    
    ontimeL17 = np.count_nonzero(L17==1)
    offtimeL17 = 64 - ontimeL17
    print("L17 min on time:", minonL17)
    print("L17 on time:", ontimeL17)
    print("L17 max off time:", maxoffL17)
    print("L17 off time:", offtimeL17)
    print("\n")
    
    ontimeL18 = np.count_nonzero(L18==1)
    offtimeL18 = 64 - ontimeL18
    print("L18 min on time:", minonL18)
    print("L18 on time:", ontimeL18)
    print("L18 max off time:", maxoffL18)
    print("L18 off time:", offtimeL18)
    print("\n")
    
    ontimeL19 = np.count_nonzero(L19==1)
    offtimeL19 = 64 - ontimeL19
    print("L19 min on time:", minonL19)
    print("L19 on time:", ontimeL19)
    print("L19 max off time:", maxoffL19)
    print("L19 off time:", offtimeL19)
    
    return

In [15]:
#print("Initial Value current cost:\n",my_swarm.current_cost)
#print("Initial Value current pos:\n",my_swarm.position)
#print("Initial Value pbest pos:\n",my_swarm.pbest_pos)
#print("Initial Value pbest cost:\n",my_swarm.pbest_cost)

#Generate SGE matrix
SGE = SGEgenerate(0)

iterations = 500 # Set 500 iterations
start = timeit.default_timer()
for i in range(iterations):
    
    # Part 0: Compute current cost
    j=0
    for j in range(my_swarm.n_particles):
        my_swarm.current_cost[j] = obj_func(my_swarm.position[j][:],SGE) # Compute current cost (using obj func)
        temp_pbest_cost[j] = obj_func(my_swarm.pbest_pos[j][:],SGE)  # Compute personal best cost (using obj func)
    
    #my_swarm.pbest_cost = f(my_swarm.pbest_pos) # Compute current cost (using built in sphere func)
    #print("-------------------------------")
    #print("Pbest pos:\n",my_swarm.pbest_pos)
    #print("Pbest cost:\n",my_swarm.pbest_cost)
    #print("Current pos:\n",my_swarm.position)
    #print("Current cost:\n", my_swarm.current_cost)
    #print("Current velocity:\n", my_swarm.velocity)
   
   
    #Update personal best pos/cost 
    my_swarm.pbest_pos, my_swarm.pbest_cost = _compute_pbest(my_swarm,temp_pbest_cost) # Update and store
    
    # Part 2: Update global best
    # Note that gbest computation is dependent on your topology
    if np.min(my_swarm.pbest_cost) < my_swarm.best_cost:
        my_swarm.best_pos, my_swarm.best_cost = my_topology.compute_gbest(my_swarm)

    # Let's print our output
    if i%100==0:
        print('Iteration: {} | my_swarm.best_cost: {:.4f}'.format(i+1, my_swarm.best_cost))
        #print("Pbest cost:\n",my_swarm.pbest_cost)
        #print("Pbest pos:\n",my_swarm.pbest_pos)
        #print("Current velocity:\n", my_swarm.velocity)
        
    # Part 3: Update position and velocity matrices
    # Note that position and velocity updates are dependent on your topology
    
    #Fix the velocity and position computation for Binary application
    my_swarm.velocity = _compute_velocity(my_swarm,c1,c2,w)
    my_swarm.position = _compute_position(my_swarm) 
    

print('The best cost found by our swarm is: {:.4f}'.format(my_swarm.best_cost))
stop = timeit.default_timer()
print('Total Runtime: ', stop - start)
#print('The best position found by our swarm is: {}'.format(my_swarm.best_pos))

#divide array into 20 loads x 64 timeslots
timeslotrange = 64
L0 = my_swarm.best_pos[0:timeslotrange]
L1 = my_swarm.best_pos[timeslotrange:(timeslotrange*2)]
L2 = my_swarm.best_pos[(timeslotrange*2):(timeslotrange*3)]
L3 = my_swarm.best_pos[(timeslotrange*3):(timeslotrange*4)]
L4 = my_swarm.best_pos[(timeslotrange*4):(timeslotrange*5)]
L5 = my_swarm.best_pos[(timeslotrange*5):(timeslotrange*6)]
L6 = my_swarm.best_pos[(timeslotrange*6):(timeslotrange*7)]
L7 = my_swarm.best_pos[(timeslotrange*7):(timeslotrange*8)]
L8 = my_swarm.best_pos[(timeslotrange*8):(timeslotrange*9)]
L9 = my_swarm.best_pos[(timeslotrange*9):(timeslotrange*10)]
L10 = my_swarm.best_pos[(timeslotrange*10):(timeslotrange*11)]
L11 = my_swarm.best_pos[(timeslotrange*11):(timeslotrange*12)]
L12 = my_swarm.best_pos[(timeslotrange*12):(timeslotrange*13)]
L13 = my_swarm.best_pos[(timeslotrange*13):(timeslotrange*14)]
L14 = my_swarm.best_pos[(timeslotrange*14):(timeslotrange*15)]
L15 = my_swarm.best_pos[(timeslotrange*15):(timeslotrange*16)]
L16 = my_swarm.best_pos[(timeslotrange*16):(timeslotrange*17)]
L17 = my_swarm.best_pos[(timeslotrange*17):(timeslotrange*18)]
L18 = my_swarm.best_pos[(timeslotrange*18):(timeslotrange*19)]
L19 = my_swarm.best_pos[(timeslotrange*19):(timeslotrange*20)]

print('Self Generated Energy:', (np.array(SGE).reshape(20,64)))
print('Schedule of Load 0:', (np.array(L0)).reshape(16,4))
print('Schedule of Load 1:', (np.array(L1)).reshape(16,4))
print('Schedule of Load 2:', (np.array(L2)).reshape(16,4))
print('Schedule of Load 3:', (np.array(L3)).reshape(16,4))
print('Schedule of Load 4:', (np.array(L4)).reshape(16,4))
print('Schedule of Load 5:', (np.array(L5)).reshape(16,4))
print('Schedule of Load 6:', (np.array(L6)).reshape(16,4))
print('Schedule of Load 7:', (np.array(L7)).reshape(16,4))
print('Schedule of Load 8:', (np.array(L8)).reshape(16,4))
print('Schedule of Load 9:', (np.array(L9)).reshape(16,4))
print('Schedule of Load 10:', (np.array(L10)).reshape(16,4))
print('Schedule of Load 11:', (np.array(L11)).reshape(16,4))
print('Schedule of Load 12:', (np.array(L12)).reshape(16,4))
print('Schedule of Load 13:', (np.array(L13)).reshape(16,4))
print('Schedule of Load 14:', (np.array(L14)).reshape(16,4))
print('Schedule of Load 15:', (np.array(L15)).reshape(16,4))
print('Schedule of Load 16:', (np.array(L16)).reshape(16,4))
print('Schedule of Load 17:', (np.array(L17)).reshape(16,4))
print('Schedule of Load 18:', (np.array(L18)).reshape(16,4))
print('Schedule of Load 19:', (np.array(L19)).reshape(16,4))
print("\n")

#checker for constraints
checkvalidity(my_swarm.best_pos,SGE)



Iteration: 1 | my_swarm.best_cost: 147925347.5000
Iteration: 101 | my_swarm.best_cost: 26610443.7500
Iteration: 201 | my_swarm.best_cost: 24936908.7500
Iteration: 301 | my_swarm.best_cost: 24896031.2500
Iteration: 401 | my_swarm.best_cost: 24896031.2500
The best cost found by our swarm is: 24896031.2500
Total Runtime:  252.47274220000003
Schedule of Load 0: [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [0 1 1 1]
 [1 0 1 1]
 [1 1 1 1]
 [0 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 0 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
Schedule of Load 1: [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 0 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
Schedule of Load 2: [[1 0 1 1]
 [1 1 1 1]
 [1 0 1 1]
 [1 1 1 0]
 [1 1 0 0]
 [0 0 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [0 1 1 1]
 [1 1 1 1]
 [0 1 1 1]
 [1 1 1 0]
 [1 1 1 0]
 [1 0 1 1]
 [1 1 0 1]
 [1 1 1 1]]
Schedule of Load 3: [[1 1 1 1]
 [0 1 0 0]
 [1 1 1 1]
 [0 1 1 1]
 [1 1